In [ ]:
# Modules used
import pandas as pd
import string
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.neural_network import MLPClassifier
import sklearn   # import countVectorizer
import nltk
nltk.download('wordnet')
nltk.download('words')
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize 

In [ ]:
nltk.download('words')
def remove_non(sent):
    words = set(nltk.corpus.words.words())
    #sent = "Io andiamo to the beach with my amico."
    sent = " ".join(w for w in nltk.wordpunct_tokenize(sent) if w.lower() in words or not w.isalpha())
    return sent

In [ ]:
# Text maker Function
def concat(text):
    # text is the form of a list
    s = ' '
    result = s.join(text)
    return result

In [ ]:
# Lemmatizer Function
def lemmatizer(text):
    ps = PorterStemmer() 
    wnl = nltk.WordNetLemmatizer()
    text = [wnl.lemmatize(word) for word in text]
    return [ps.stem(word) for word in text]

In [ ]:
# cleaner Function
def cleanText(textRecords):
    count = 0
    #return this 
    cleanedText = []
    

    for text in textRecords:
    
        # regex to remove all and any links
        URL_REGEX = r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"      
        result = re.sub(URL_REGEX, '', text, flags=re.MULTILINE)   
        
        # Remove Numbers 
        result = ''.join([i for i in result if not i.isdigit()])
        
         # to upper
        result = result.lower() 
    
        # make list of words
        words = result.split() 
        print(words)
        # removes all puntuation marks
        #table = str.maketrans('', '', string.punctuation)          
        #stripped = [w.translate(table) for w in words]
    
        count += 1 # keeping track of number of conversions
        stripped = words
        # remove this character and empty spaces from the list
        for i in stripped:     
            if '\x89Û' in i or ' ' in i or '&amp;' in i:
                stripped.remove(i)  
                
                
        # retain root words
        stripped = lemmatizer(stripped)
        
        # remove non english words
        final = remove_non(concat(stripped))
        
        # Concatenate strings to form cleaned text
        cleanedText.append(final)
        
        print(count)
    return cleanedText


In [ ]:
# clean Training data
disasterTrain = pd.read_csv('train.csv')
disasterTrain.drop(columns = ['keyword', 'location'], inplace=True)

textRecords_train = disasterTrain['text']
cleanRecords_train = cleanText(textRecords_train)
cleanRecords_train = pd.Series(cleanRecords_train)
disasterTrain['CleanedText'] = cleanRecords_train
disasterTrain.drop(columns = 'text', inplace=True)

print(disasterTrain) 


In [ ]:
# clean Testing data
disasterTest = pd.read_csv('test.csv')
disasterTest.drop(columns = ['keyword', 'location'], inplace=True)

textRecords_test = disasterTest['text']
cleanRecords_test = cleanText(textRecords_test)
cleanRecords_test = pd.Series(cleanRecords_test)
disasterTest['CleanedText'] = cleanRecords_test
disasterTest.drop(columns = 'text', inplace=True)

print(disasterTest)


In [ ]:
# Vectorise Text(train) input!
import sklearn   # import countVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
count_vectorizer = feature_extraction.text.CountVectorizer(strip_accents = 'ascii',stop_words = 'english')

disasterVectors_train = count_vectorizer.fit_transform(disasterTrain["CleanedText"])
tfidf_trnsfrmr = TfidfTransformer(use_idf=False)
tfidf_train = tfidf_trnsfrmr.fit_transform(disasterVectors_train)
disasterVectors_test = tfidf_trnsfrmr.transform(count_vectorizer.transform(disasterTest["CleanedText"]))
disasterVectors_test = count_vectorizer.transform(disasterTest["CleanedText"])

print(disasterVectors_train[0].todense().shape)
print(disasterVectors_train[0].todense())

print(disasterVectors_test[0].todense().shape)
print(disasterVectors_test[0].todense())

#### Random Forest Classifier

In [7]:
from sklearn.ensemble import RandomForestClassifier


X_train,X_test,y_train,y_test = train_test_split(disasterVectors_train,disasterTrain.target,random_state=42,stratify=disasterTrain.target)
n_estimators = range(,30)
result_list2 = []
for n_estimator in n_estimators:
    forest = RandomForestClassifier(n_estimators=n_estimator,random_state=2,n_jobs=-1)
    forest.fit(X_train,y_train)
    forest_list = [n_estimator,round(forest.score(X_train,y_train),3),round(forest.score(X_test,y_test),3)]
    result_list2.append(forest_list)
for i in result_list2:
    print(i,'\n')
#print(f"Accuracy:\t{round(forest.score(X_train,y_train),3)}")
#print(f"Accuracy:\t{round(forest.score(X_test,y_test),3)}")




NameError: name 'disasterVectors' is not defined

#### Gradient Boosting Classifier

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(disasterVectors,disasterTrain.target,random_state=42,stratify=disasterTrain.target,test_size=0.1,train_size=0.9)

from sklearn.ensemble import GradientBoostingClassifier
learning_rates = [0.01,0.03,0.1,0.3,1]
depths = range(1,10)
result_list = []
for learning_rate in learning_rates:
    for depth in depths:
        gbrt = GradientBoostingClassifier(random_state=0,learning_rate=learning_rate,max_depth=depth)
        gbrt.fit(X_train,y_train)
        for_list = [learning_rate,depth,round(gbrt.score(X_train,y_train),3),round(gbrt.score(X_test,y_test),3)]
        result_list.append(for_list)
        
for i in result_list:
    print(i,'\n')
  


In [ ]:
sample_submission = pd.read_csv("sample_submission.csv")
sample_submission

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
learning_rates = []
depths = [4]


for learning_rate in learning_rates:
    for depth in depths:
        for_list = []
        gbrt = GradientBoostingClassifier(random_state=0,learning_rate=learning_rate,max_depth=depth)
        gbrt.fit(disasterVectors_train,disasterTrain.target)
        sample_submission["target"] = gbrt.predict(disasterVectors_test)

In [ ]:
sample_submission

In [ ]:
# save the dataframe locally
sample_submission.to_csv (r'C:\Users\Akshay Desai\Desktop\All Projects\Real or Not\sample_submission2.csv', index = False, header=True)

In [ ]:
scores = model_selection.cross_val_score(gbrt, disasterVectors_train,disasterTrain.target, cv=3,scoring='f1')
scores

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV


parameters = {'learning_rate':[0.01,0.03,0.1,0.3,1],'max_depth' :list(range(1,10))}

gbrt = GradientBoostingClassifier()

clf = GridSearchCV(gbrt, parameters,verbose=1)
clf.fit(disasterVectors_train,disasterTrain.target)

sample_submission["target"] = clf.predict(disasterVectors_test)

## Multilayer Perceptrons classifier

In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html
from sklearn.model_selection import GridSearchCV

layers = [[1000],[2500],[5000],[10000],[100,100],[1000,1000],[2500,2500]]
solver = ['lbfgs','adam','sgd']
alpha = [0.001,0.003,0.01,0.03,0.1,0.3,1]
activation = ['identity','logistic','tanh','relu']

parameters = {'hidden_layer_sizes':layers,'solver':solver,'alpha':alpha,'activation':activation}

mlp = MLPClassifier(max_iter=1000,random_state=42)

clf = GridSearchCV(mlp,parameters,verbose=1)
clf.fit(disasterVectors_train,disasterTrain.target)

sample_submission["target"] = clf.predict(disasterVectors_test)

Fitting 5 folds for each of 588 candidates, totalling 2940 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [17]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

parameters = {'n_estimators':list(range(100,500))}

forest = RandomForestClassifier(random_state=42,n_jobs=-1)
clf = GridSearchCV(mlp,parameters,verbose=1)
clf.fit(disasterVectors_train,disasterTrain.target)

sample_submission["target"] = clf.predict(disasterVectors_test)

NameError: name 'GridSearchCV' is not defined